# Dataset generator

In [10]:
import os
def list_files(directory):
    ret = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            #print(os.path.join(root, file))
            ret.append(os.path.join(root, file))
    return ret
list_files('E:\\HanlinMath\\ElementarySchool\\grade1')

[]

In [11]:
from docx import Document

def generate_dataset(file_path):
    print(file_path)
    doc = Document(file_path)
    #print('段落數量： ', len(doc.paragraphs))
    idx = 0
    dataset = []
    chapter = doc.paragraphs[2].text
    while idx < len(doc.paragraphs):
        
        if doc.paragraphs[idx].text[:3] != "編號：": 
            idx += 1
            continue
        question = {}
        input, output, difficulty = "", "", doc.paragraphs[idx].text.split("難易度：")[-1]
        idx += 1
        while idx < len(doc.paragraphs) and doc.paragraphs[idx].text[:3] != "答案：":
            input += doc.paragraphs[idx].text
            idx += 1
        while idx < len(doc.paragraphs) and doc.paragraphs[idx].text[:5] != "認知向度：":
            output += doc.paragraphs[idx].text + " "
            idx += 1
        question["input"] = input
        question["output"] = output
        question["difficulty"] = difficulty
        question["chapter"] = chapter
        dataset.append(question)
        
    return dataset

generate_dataset("E:\\HanlinMath\\ElementarySchool\\grade1_2\\1_1.docx")

E:\HanlinMath\ElementarySchool\grade1_2\1_1.docx


[{'input': '(   )算算看，哪一個答案不是\u300015？\u3000(Ａ)\u30009＋6\u3000(Ｂ)\u30008＋7\u3000(Ｃ)\u30006＋8。',
  'output': '答案：(Ｃ) 解析：(Ａ)\u30009＋6＝15；(Ｂ)\u30008＋7＝15；(Ｃ)\u30006＋8＝14 ',
  'difficulty': '中',
  'chapter': '1-1_基本加法'},
 {'input': '(   )包子店上午賣出\u30005\u3000個高麗菜包，下午賣出\u30009\u3000個高麗菜包，包子店共賣出幾個高麗菜包？\u3000(Ａ)\u300012\u3000個\u3000(Ｂ)\u300013\u3000個\u3000(Ｃ)\u300014\u3000個。',
  'output': '答案：(Ｃ) 解析：5＋9＝14 ',
  'difficulty': '易',
  'chapter': '1-1_基本加法'},
 {'input': '(   )籃子裡有\u30005\u3000顆綠球和\u30007\u3000顆黃球，籃子裡共有幾顆球？\u3000(Ａ)\u300013\u3000顆\u3000(Ｂ)\u300012\u3000顆\u3000(Ｃ)\u300011\u3000顆。',
  'output': '答案：(Ｂ) 解析：5＋7＝12 ',
  'difficulty': '易',
  'chapter': '1-1_基本加法'},
 {'input': '(   )操場上原有\u30009\u3000個小朋友，又來\u30006\u3000個小朋友，操場上現在共有幾個小朋友？\u3000(Ａ)\u300015\u3000個\u3000(Ｂ)\u300016\u3000個\u3000(Ｃ)\u300017\u3000個。',
  'output': '答案：(Ａ) 解析：9＋6＝15 ',
  'difficulty': '易',
  'chapter': '1-1_基本加法'},
 {'input': '(   )書桌上有\u300010\u3000本名人傳記和\u30006\u3000本科學小百科，書桌上一共有幾本書？\u3000(Ａ)\u300015\u3000本\u300

In [12]:
import csv
def write_csv(datasets, file_path):
    with open(file_path, 'w+', encoding='utf-8') as f:
        fieldnames = ['input', 'output', 'difficulty', 'chapter']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for dataset in datasets:
            for data in dataset:
                writer.writerow(data)
    print('Save data in ' + file_path)

In [13]:
file_path = 'E:\\HanlinMath\\ElementarySchool\\grade1_1'
files = list_files(file_path)
datasets = []
for file in files:
    datasets.append(generate_dataset(file))

save_path = 'dataset\\' + file_path.split('\\')[-1] + '.csv'
write_csv(datasets, save_path)

E:\HanlinMath\ElementarySchool\grade1_1\3_1.docx
E:\HanlinMath\ElementarySchool\grade1_1\3_2.docx
E:\HanlinMath\ElementarySchool\grade1_1\3_3.docx
E:\HanlinMath\ElementarySchool\grade1_1\4_4.docx
E:\HanlinMath\ElementarySchool\grade1_1\6_1.docx
E:\HanlinMath\ElementarySchool\grade1_1\6_3.docx
E:\HanlinMath\ElementarySchool\grade1_1\7_1.docx
E:\HanlinMath\ElementarySchool\grade1_1\7_2.docx
E:\HanlinMath\ElementarySchool\grade1_1\7_3.docx
E:\HanlinMath\ElementarySchool\grade1_1\7_4.docx
E:\HanlinMath\ElementarySchool\grade1_1\8_2.docx
E:\HanlinMath\ElementarySchool\grade1_1\8_4.docx
E:\HanlinMath\ElementarySchool\grade1_1\9_1.docx
Save data in dataset\grade1_1.csv


In [15]:
from langsmith import Client
client = Client()
file = "grade1_1"

dataset = client.upload_csv(
    csv_file = f"dataset\\{file}.csv",
    input_keys = ['input', 'chapter', 'difficulty'],
    output_keys = ['output'],
    name = file,
    description = f"Hanlin Math of {file}",
    data_type = "kv"
)

In [25]:
import langsmith
from langchain import smith
from langchain_openai import ChatOpenAI
from src.dependencies.settings import get_settings
from src.service.math_solver import MathSolver

# Define your runnable or chain below.
math_solver = MathSolver(model="gpt-4-1106-preview", temperature=0)

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa",
        smith.RunEvalConfig.LabeledCriteria("helpfulness"),
        smith.RunEvalConfig.LabeledCriteria("conciseness")
    ],
    custom_evaluators=[],
    eval_llm = ChatOpenAI(model="gpt-3.5-turbo", 
        temperature=0, 
        openai_api_key=get_settings().gpt_secret_key
    )
)

client = langsmith.Client()

def run(dataset):
    return client.run_on_dataset(
        dataset_name=dataset,
        llm_or_chain_factory=math_solver.agent,
        evaluation=eval_config,
        project_name="QSticker Math APP evaluation",
        concurrency_level=5,
        verbose=True,
    )

In [34]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate
math_solver = MathSolver(model="gpt-4-1106-preview", temperature=0)
eval_llm = ChatOpenAI(model="gpt-3.5-turbo", 
    temperature=0, 
    openai_api_key=get_settings().gpt_secret_key
)
qa_evaluator = LangChainStringEvaluator(
    "qa", 
    config={"llm": eval_llm},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["output"],
        "input": example.inputs["input"],
    }
)  
context_qa_evaluator = LangChainStringEvaluator(
    "context_qa", 
    config={"llm": eval_llm},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["output"],
        "input": example.inputs["input"],
    }
)
cot_qa_evaluator = LangChainStringEvaluator(
    "cot_qa", 
    config={"llm": eval_llm},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["output"],
        "input": example.inputs["input"],
    }
)

def run_evaluate(dataset):
    return evaluate(
        lambda x: math_solver.agent.invoke(x),
        data=dataset,
        evaluators = [qa_evaluator, context_qa_evaluator, cot_qa_evaluator],
        experiment_prefix="Hanlin Math Elementary " + dataset,
    )

In [35]:
run_evaluate("grade1_1")

View the evaluation results for experiment: 'Hanlin Math Elementary grade1_1-73745275' at:
https://smith.langchain.com/o/0c9779fc-55de-54eb-b86b-a76ff39ff60c/datasets/a9ddf5d7-9c9e-4584-a7cc-272e9de7d6a8/compare?selectedSessions=0b8502a1-cf24-4e48-9179-06371dc789a4




0it [00:00, ?it/s]Error running target function: unknown format from LLM: The `numexpr` library in Python is not designed to sort numbers. It is used for fast numerical evaluations of array expressions, mainly arithmetic operations. Sorting is typically done using Python's built-in functions like `sorted()` or methods like `.sort()` for lists.

However, if you want to sort the numbers 29, 7, and 12 using Python (not specifically with `numexpr`), you can do it like this:

```python
sorted([29, 7, 12])
```

This will give you the sorted list `[7, 12, 29]`. If you need to use `numexpr` for some reason, you would have to sort the numbers outside of the `numexpr.evaluate()` call.
Error running target function: Got unsupported early_stopping_method `generate`
Error running target function: Got unsupported early_stopping_method `generate`
54it [04:07,  4.58s/it]


<ExperimentResults Hanlin Math Elementary grade1_1-73745275>